In [1]:
install.packages("tidyverse")
install.packages("caTools")
install.packages("pROC")
install.packages("ROCR")
install.packages("rpart")
install.packages("rpart.plot")
install.packages("randomForest")

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
also installing the dependency ‘stringr’

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


In [2]:
# Import dataset
stevens = read.csv("./files/stevens.csv")
str(stevens)
head(stevens)

'data.frame':	566 obs. of  9 variables:
 $ Docket    : Factor w/ 566 levels "00-1011","00-1045",..: 63 69 70 145 97 181 242 289 334 436 ...
 $ Term      : int  1994 1994 1994 1994 1995 1995 1996 1997 1997 1999 ...
 $ Circuit   : Factor w/ 13 levels "10th","11th",..: 4 11 7 3 9 11 13 11 12 2 ...
 $ Issue     : Factor w/ 11 levels "Attorneys","CivilRights",..: 5 5 5 5 9 5 5 5 5 3 ...
 $ Petitioner: Factor w/ 12 levels "AMERICAN.INDIAN",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ Respondent: Factor w/ 12 levels "AMERICAN.INDIAN",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ LowerCourt: Factor w/ 2 levels "conser","liberal": 2 2 2 1 1 1 1 1 1 1 ...
 $ Unconst   : int  0 0 0 0 0 1 0 1 0 0 ...
 $ Reverse   : int  1 1 1 1 1 0 1 1 1 1 ...


Docket,Term,Circuit,Issue,Petitioner,Respondent,LowerCourt,Unconst,Reverse
93-1408,1994,2nd,EconomicActivity,BUSINESS,BUSINESS,liberal,0,1
93-1577,1994,9th,EconomicActivity,BUSINESS,BUSINESS,liberal,0,1
93-1612,1994,5th,EconomicActivity,BUSINESS,BUSINESS,liberal,0,1
94-623,1994,1st,EconomicActivity,BUSINESS,BUSINESS,conser,0,1
94-1175,1995,7th,JudicialPower,BUSINESS,BUSINESS,conser,0,1
95-129,1995,9th,EconomicActivity,BUSINESS,BUSINESS,conser,1,0


In [3]:
# Divide the dataset into training, and testing subsets
library(caTools)
set.seed(3000)
split = sample.split(stevens$Reverse, .7)
train = subset(stevens, split)
test =  subset(stevens, !split)

In [4]:
# Build tree model
library(rpart)
library(rpart.plot)
stevensTree = rpart(formula=Reverse ~ Circuit + Issue + Petitioner + Respondent + LowerCourt + Unconst, data=train, 
                    method="class", minbucket = 25)

In [5]:
plot(stevensTree)
prp(stevensTree)
rpart.plot(stevensTree)

In [6]:
# Checking model with test datatest
predictTest = predict(stevensTree, newdata=test, type="class")
tbl = table(test$Reverse, predictTest)
print(tbl)
# Baseline accuracy is always predicting no heart disease at 10 years
baselineacc = (tbl[2]+tbl[4]) / (tbl[1]+tbl[2]+tbl[3]+tbl[4])
# Model accuracy
acc = (tbl[1]+tbl[4]) / (tbl[1]+tbl[2]+tbl[3]+tbl[4])

print(baselineacc)
print(acc)

   predictTest
     0  1
  0 41 36
  1 22 71
[1] 0.5470588
[1] 0.6588235


In [11]:
# Check ROC, method 1
library(pROC)
predicted = predict(stevensTree, newdata=test)
roc = roc(test$Reverse, predicted[,2])
auc(roc)
plot(roc)

Area under the curve: 0.6927

In [14]:
# Check ROC, method 2
library(ROCR)
predicted = predict(stevensTree, newdata=test)
preds = prediction(predicted[,2], test$Reverse)
perf1 = performance(preds, "auc")
perf2 = performance(preds, "tpr", "fpr")
print(perf1@y.values)
plot(perf2)

[[1]]
[1] 0.6927105

